https://www.scribendi.ai/comparing-bert-and-gpt-2-as-language-models-to-score-the-grammatical-correctness-of-a-sentence/

# Installing Java-8

In [ ]:
import os

In [ ]:
!apt-get purge openjdk-\* icedtea-\* icedtea6-\*
!apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!java -version

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'openjdk-9-jre-headless' for glob 'openjdk-*'
Note, selecting 'openjdk-8-jdk' for glob 'openjdk-*'
Note, selecting 'openjdk-8-jre' for glob 'openjdk-*'
Note, selecting 'openjdk-6-jre' for glob 'openjdk-*'
Note, selecting 'openjdk-6-jre-headless' for glob 'openjdk-*'
Note, selecting 'openjdk-11-demo' for glob 'openjdk-*'
Note, selecting 'openjdk-8-demo' for glob 'openjdk-*'
Note, selecting 'openjdk-11-source' for glob 'openjdk-*'
Note, selecting 'openjdk-8-jre-dcevm' for glob 'openjdk-*'
Note, selecting 'openjdk-11-jre-headless' for glob 'openjdk-*'
Note, selecting 'openjdk-11-dbg' for glob 'openjdk-*'
Note, selecting 'openjdk-11-doc' for glob 'openjdk-*'
Note, selecting 'openjdk-8-jdk-headless' for glob 'openjdk-*'
Note, selecting 'openjdk-7-jre-headless' for glob 'openjdk-*'
Note, selecting 'openjdk-8-jre-zero' for glob 'openjdk-*'
Note, selecting 'openjdk-8-source' for glob

# Importing Libraries

In [ ]:
!pip install transformers
!pip install language_check

     |████████████████████████████████| 2.1MB 5.7MB/s 
     |████████████████████████████████| 901kB 35.7MB/s 
     |████████████████████████████████| 3.3MB 34.7MB/s 
  Created wheel for language-check: filename=language_check-1.1-cp37-none-any.whl size=90190897 sha256=6c46fe6422bfe85d2b5fda6ada77917fa9c484a17c0612b9a99e1d9f5c7190ea
  Stored in directory: /root/.cache/pip/wheels/d5/46/82/90a89c23eac1837364ed7217a9eed71bc9e6ad4825be93968e
Successfully built language-check


In [ ]:
import json
import torch
import math
import numpy as np
import pandas as pd
import language_check
import matplotlib.pyplot as plt
from google.colab import files
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

In [ ]:
tool_us = language_check.LanguageTool('en-US')
tool_uk = language_check.LanguageTool('en-GB')

# Reading Dataset

In [12]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/haiku/results.tsv", sep='\t')

In [13]:
print(df.shape)
df.head()

(37797, 4)


,model,type,poem,haiku
0,bert-clean,stf-bert-test-clean,"another thought comes to mind, before it can b...","thought comes thoughts, sleepless"
1,bert-clean,stf-bert-test-clean,homework is such a bore others never do it m y...,homework others a ton
2,bert-clean,stf-bert-test-clean,"i had a gig-horse, and i called him pleasure b...",sundays a little jaunt
3,bert-clean,stf-bert-test-clean,"the allegory - 101 march 7, 2019 the forest is...","allegory the small full ferocious animals,"
4,bert-clean,stf-bert-test-clean,"the allegory - 161 march 18, 2019 the state is...",allegory the state full ferocious animals all ...


# Defining Functions

In [14]:
def perplexityGPT2(sentences):
  model_id = 'gpt2'
  model = GPT2LMHeadModel.from_pretrained(model_id)
  tokenizer = GPT2TokenizerFast.from_pretrained(model_id)

  ppl = list()
  total_length = len(sentences)
  for index, sent in enumerate(sentences):
    if not (index+1)%1000:
      print(f"{index+1}/{total_length}")
    tokenize_input = tokenizer.encode(sent)
    tensor_input = torch.tensor([tokenize_input])
    loss = model(tensor_input, labels=tensor_input)[0]
    ppl.append(math.exp(loss))

  return ppl

In [15]:
def checkGrammar(sentences):
  result = list()
  total_length = len(sentences)
  for index, sent in enumerate(sentences):
    if not (index+1)%1000:
      print(f"{index+1}/{total_length}")
    flag_us = not bool(tool_us.check(sent))
    flag_uk = not bool(tool_uk.check(sent))
    result.append(flag_us or flag_uk)
  return result

# Calculating Perplexity

In [ ]:
ppl_scores_gpt2 = perplexityGPT2(list(df["haiku"].values))
df["ppl-gpt2"] = ppl_scores_gpt2

# Checking Grammar

In [ ]:
grammar_checks = checkGrammar(list(df["haiku"].values))
df["grammar-check"] = grammar_checks

1000/54629
2000/54629
3000/54629
4000/54629
5000/54629
6000/54629
7000/54629
8000/54629


KeyboardInterrupt: ignored

# Downloading Dataset

In [ ]:
df.to_json("ppl-grammar-output.json")
df.to_csv("ppl-grammar-output.csv", index=False, sep='\t')

In [ ]:
df.to_json("/content/drive/MyDrive/Colab Notebooks/ppl-grammar-output.json")
df.to_csv("/content/drive/MyDrive/Colab Notebooks/ppl-grammar-output.csv", index=False, sep='\t')

In [ ]:
files.download("ppl-grammar-output.json")
files.download("ppl-grammar-output.csv")